In [1]:
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

import dataset
import schedulers

import tensorflow as tf
assert tf.config.list_physical_devices('GPU')

In [2]:
BATCH_SIZE = 64

LETTERS_SIZE = len(dataset.letters_table)
NIQQUD_SIZE = len(dataset.niqqud_table)
DAGESH_SIZE = len(dataset.dagesh_table)
SIN_SIZE = len(dataset.sin_table)

def build_model(UNITS=500):
    inp = keras.Input(shape=(None,), batch_size=None)
    embed = layers.Embedding(LETTERS_SIZE, UNITS, mask_zero=True)(inp)
    
    layer = layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.0), merge_mode='sum')(embed)
    layer = layer + layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.0), merge_mode='sum')(layer)
    layer = layers.BatchNormalization()(layer)
    layer = embed + layers.Dense(UNITS, activation='relu')(layer)

    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    model = keras.Model(inputs=inp, outputs=outputs)

    return model

model = build_model()

model.summary()
model.save_weights('./checkpoints/uninit')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 500)    22000       input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, None, 500)    4004000     embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, None, 500)    4004000     bidirectional[0][0]              
______________________________________________________________________________________________

In [3]:

# masked version of accuracy and sce
def accuracy(real, pred):
    acc = tf.keras.metrics.sparse_categorical_accuracy(real, pred)

    mask = tf.cast(tf.math.logical_not(tf.math.equal(real, 0)), dtype=acc.dtype)
    acc *= mask

    return tf.reduce_sum(acc) / tf.reduce_sum(mask)

def sparse_categorical_crossentropy(y_true, y_pred, sample_weight=None):
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)

    mask = tf.cast(tf.math.logical_not(tf.math.equal(y_true, 0)), dtype=loss.dtype)
    loss *= mask

    return tf.reduce_sum(loss) / tf.reduce_sum(mask) 

def fit(train_validation, scheduler=None, verbose=1, lr=1e-4, epochs=1):
    train, valid = train_validation
    model.compile(loss=sparse_categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  metrics={'N': accuracy, 'D': accuracy, 'S': accuracy})
    callbacks = []
    if isinstance(scheduler, schedulers.CircularLearningRate):
        scheduler.set_dataset(train, BATCH_SIZE)
    if scheduler:
        callbacks.append(scheduler)
        
    x  = train.normalized
    vx = valid.normalized
    
    y  = {'N': train.niqqud, 'D': train.dagesh, 'S': train.sin }
    vy = {'N': valid.niqqud, 'D': valid.dagesh, 'S': valid.sin }
    
    return model.fit(x, y, validation_data=(vx, vy), batch_size=BATCH_SIZE, epochs=epochs, verbose=verbose, callbacks=callbacks)


MAXLEN = 64
def load_data(source, maxlen=MAXLEN, validation=0.1):
    filenames = [os.path.join('texts', f) for f in source]
    train, valid = dataset.load_data(filenames, validation, maxlen=maxlen)
    return train, valid

In [4]:
data_mix = load_data(['poetry', 'rabanit', 'pre_modern'])

In [5]:
data_modern = load_data(validation=0.2, source=['modern'])

In [6]:
model.load_weights('./checkpoints/uninit')
history = fit(data_mix, scheduler=schedulers.CircularLearningRate(30e-4, 80e-4, 1e-4), epochs=1)
model.save_weights('./checkpoints/mix')

2184/2184 [==============================] - 235s 107ms/step - loss: 0.2693 - N_loss: 0.2061 - D_loss: 0.0560 - S_loss: 0.0071 - N_accuracy: 0.9317 - D_accuracy: 0.9789 - S_accuracy: 0.9979 - val_loss: 0.2184 - val_N_loss: 0.1656 - val_D_loss: 0.0487 - val_S_loss: 0.0040 - val_N_accuracy: 0.9474 - val_D_accuracy: 0.9812 - val_S_accuracy: 0.9989


In [7]:
model.load_weights('./checkpoints/mix')
history = fit(data_modern, scheduler=schedulers.CircularLearningRate(50e-4, 50e-4, 1e-5))
model.save_weights('./checkpoints/modern')

234/234 [==============================] - 30s 129ms/step - loss: 0.1594 - N_loss: 0.1191 - D_loss: 0.0378 - S_loss: 0.0026 - N_accuracy: 0.9622 - D_accuracy: 0.9866 - S_accuracy: 0.9994 - val_loss: 0.1492 - val_N_loss: 0.1120 - val_D_loss: 0.0357 - val_S_loss: 0.0016 - val_N_accuracy: 0.9645 - val_D_accuracy: 0.9877 - val_S_accuracy: 0.9996


In [8]:
model.load_weights('./checkpoints/modern')
history = fit(data_modern, scheduler=schedulers.CircularLearningRate(5e-4, 15e-4, 1e-5), epochs=1)
history = fit(data_modern, scheduler=schedulers.CircularLearningRate(5e-5, 1e-4, 1e-5), epochs=1)
model.save_weights('./checkpoints/modern_over')

234/234 [==============================] - 30s 129ms/step - loss: 0.0303 - N_loss: 0.0209 - D_loss: 0.0089 - S_loss: 4.9314e-04 - N_accuracy: 0.9938 - D_accuracy: 0.9971 - S_accuracy: 0.9999 - val_loss: 0.1556 - val_N_loss: 0.1155 - val_D_loss: 0.0385 - val_S_loss: 0.0016 - val_N_accuracy: 0.9687 - val_D_accuracy: 0.9887 - val_S_accuracy: 0.9996accu


In [ ]:
model.load_weights('./checkpoints/modern')
model.save('modern.h5')

In [ ]:
import tensorflowjs as tfjs
model.load_weights('./checkpoints/modern')
tfjs.converters.save_keras_model(model, '.')

In [9]:
model.load_weights('./checkpoints/modern')

def print_predictions(data, s):
    batch = data.normalized[s]
    prediction = model.predict(batch)
    [actual_niqqud, actual_dagesh, actual_sin] = [dataset.from_categorical(prediction[0]), dataset.from_categorical(prediction[1]), dataset.from_categorical(prediction[2])]
    [expected_niqqud, expected_dagesh, expected_sin] = [data.niqqud[s], data.dagesh[s], data.sin[s]]
    actual = dataset.merge(data.text[s], ts=batch, ns=actual_niqqud, ds=actual_dagesh, ss=actual_sin)
    expected = dataset.merge(data.text[s], ts=batch, ns=expected_niqqud, ds=expected_dagesh, ss=expected_sin)
    total = []
    for i, (a, e) in enumerate(zip(actual, expected)):
        print('מצוי: ', a)
        print('רצוי: ', e)
        last = expected_niqqud[i].tolist().index(0)
        res = expected_niqqud[i][:last] == actual_niqqud[i][:last]
        total.extend(res)
        print(round(np.mean(res), 2), f'({last - sum(res)} out of {last})')
        print()
    print(round(np.mean(total), 3))

print_predictions(data_modern[1], slice(0, None))

מצוי:  בִּמְיוּחָד וּמְאוּפְייְנוֹת בְּאֵלֵמֶנְטִים שֶׁל תְּנוּחוֹת מִינִיּוּת הַכּוֹלְלוֹת 
רצוי:  בִּמְיוּחָד וּמְאוּפְיָינוֹת בְּאֵלֵמֶנְטִים שֶׁל תְּנוּחוֹת מִינִיּוֹת הַכּוֹלְלוֹת 
0.94 (3 out of 52)

מצוי:  בְּאַחַת מִן הַדְּרָכִים הַמְּרוּבּוֹת הַפְּתוּחוֹת לִפְנֵיהֶם. עַל חַיָּילִים בְּשֵׁירוּת סָדִיר 
רצוי:  בְּאַחַת מִן הַדְּרָכִים הַמְּרוּבּוֹת הַפְּתוּחוֹת לִפְנֵיהֶם. עַל חַיָּילִים בְּשֵׁירוּת סָדִיר 
1.0 (0 out of 61)

מצוי:  לְהִתְנַהֵג בְּעַצְבָּנוּת. כְּשֶׁהִגַּעְנוּ הוּא יָצָא רִאשׁוֹן וּבְקוֹשִׁי הִסְתַּכֵּל עָלַיי. שָׁאַלְתִּי 
רצוי:  לְהִתְנַהֵג בְּעַצְבָּנוּת. כְּשֶׁהִגַּעְנוּ הוּא יָצָא רִאשׁוֹן וּבַקּוֹשִׁי הִסְתַּכֵּל עָלַיי. שָׁאַלְתִּי 
0.98 (1 out of 63)

מצוי:  הַלְּחִימָה, בְּעִקְבוֹת כְּנִיסָתָהּ לְתוֹקֶף שֶׁל הַפְסָקַת אֵשׁ שֶׁהוּשְּׂגָה בְּחָסוּת מוֹעֶצֶת 
רצוי:  הַלְּחִימָה, בְּעִקְבוֹת כְּנִיסָתָהּ לְתוֹקֶף שֶׁל הַפְסָקַת אֵשׁ שֶׁהוּשְּׂגָה בְּחָסוּת מוֹעֶצֶת 
1.0 (0 out of 59)

מצוי:  יְקָרִים עַל מַאֲבָקֵי כּוֹחַ יַלְדוּתִיִּים בֵּין מַעֲרֶכֶת הַבְּ

In [ ]:
for i in range(50):
    model.load_weights('./checkpoints/pre_modern')
    p1 = np.exp(np.random.uniform(low=np.log(1e-5), high=np.log(1e-2)))
    p2 = np.exp(np.random.uniform(low=np.log(1e-4), high=np.log(1e-1)))
    p3 = np.exp(np.random.uniform(low=np.log(1e-5), high=np.log(1e-2)))
    print(p1, p2, p3, end=', ', sep=', ')
    history = fit(data_modern, scheduler=schedulers.CircularLearningRate(p1, p2, p3), verbose=0)
    print(history.history['val_N_accuracy'][0])

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=3)

for n, v in enumerate(['accuracy', 'loss'], 0):
    for n1, t in enumerate(['N', 'D', 'S'], 0):
        p = ax[n][n1]
        p.plot(history.history[t + '_' + v][0:])
        p.plot(history.history['val_' + t + '_' +  v][0:])
        p.legend([t + '_Train', t + '_Test'], loc='center right')

plt.tight_layout()